### Ньяти Каелиле БВТ2201
### Курсовая Работа - СИАОД

#### 1. В Лоб - Brute Force Алгоритм 

In [40]:
from datetime import datetime, timedelta

# Constants
WEEKDAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
WEEKENDS = ['Saturday', 'Sunday']
SHIFT_8_HOURS = timedelta(hours=8)
SHIFT_12_HOURS = timedelta(hours=12)

# Helper function to create time blocks for shifts
def create_time_blocks(start_time, shift_duration):
    start = datetime.strptime(start_time, "%H:%M")
    end = start + shift_duration
    return start, end

# Helper function to assign breaks based on the shift type
def assign_breaks(driver, start, end, break_type):
    """Assign breaks to the driver during their shift."""
    breaks = []
    if break_type == "15-min":
        # Stagger 15-min breaks every 1hr 45 mins
        current = start + timedelta(minutes=105)

        if driver.shift_type == "12-hour":
            break_duration = timedelta(minutes=18)
            while current + break_duration < end:
                breaks.append((current.strftime("%H:%M"), (current + break_duration).strftime("%H:%M"), "15-min"))
                current += timedelta(minutes=120)
        else:
            break_duration = timedelta(minutes=15)
            while current + break_duration < end:
                breaks.append((current.strftime("%H:%M"), (current + break_duration).strftime("%H:%M"), "15-min"))
                current += timedelta(minutes=120)

    elif break_type == "1-hour":
        if start.strftime("%H:%M") == "09:00":
            lunch_start = datetime.strptime("14:15", "%H:%M")
            lunch_end = lunch_start + timedelta(minutes=45)
        else:
            lunch_start = datetime.strptime("13:00", "%H:%M")
            lunch_end = lunch_start + timedelta(hours=1)

        if start <= lunch_start < end:
            breaks.append((lunch_start.strftime("%H:%M"), lunch_end.strftime("%H:%M"), "1-hour"))
        elif lunch_start <= start < lunch_end:
            breaks.append((start.strftime("%H:%M"), (start + timedelta(hours=1)).strftime("%H:%M"), "1-hour"))

    return breaks


# Driver class
class Driver:
    def __init__(self, driver_id, shift_type):
        self.driver_id = driver_id
        self.shift_type = shift_type
        self.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
        self.breaks = {day: [] for day in WEEKDAYS + WEEKENDS}

# Bus class
class Bus:
    def __init__(self, bus_id, bus_index):
        self.bus_id = bus_id
        self.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
        self.drivers = []  # List of drivers assigned to this bus
        self.bus_index = bus_index  # Add this attribute

# Create buses with unique indices
buses = [Bus(f"Bus_{i+1}", i) for i in range(8)]


# Create drivers (8-hour and 12-hour)
driver_count = 0
drivers = []

# Rest day counter for 12-hour drivers
rest_day_counter = {}

# Reset schedules and rest-day counter
for bus in buses:
    bus.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}

for driver in drivers:
    driver.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
    driver.breaks = {day: [] for day in WEEKDAYS + WEEKENDS}

# Allocate 8-hour drivers
def allocate_8hr_drivers():
    global driver_count

    for bus_index, bus in enumerate(buses):
        # Assign the first 4 buses to the 06:00 start group
        if bus_index < 4:
            shift_start_time = "06:00"
        else:  # Assign the last 4 buses to the 09:00 start group
            shift_start_time = "09:00"

        driver_count += 1
        driver = Driver(f"Driver_{driver_count}_8hr", "8-hour")
        drivers.append(driver)

        for day in WEEKDAYS:  # 8-hour drivers only work on weekdays
            # Stagger start times for buses in each group
            staggered_start_time = (
                datetime.strptime(shift_start_time, "%H:%M") + timedelta(minutes=(bus_index % 4) * 15)
            ).strftime("%H:%M")
            start, end = create_time_blocks(staggered_start_time, SHIFT_8_HOURS)

            # Assign breaks: Always assign a lunch break for each day
            if "09:00" in staggered_start_time:
                # Drivers starting at 09:00 get a lunch break
                breaks = assign_breaks(driver, start, end, "1-hour")
            else:
                # Drivers starting at 06:00 get 15-minute breaks
                breaks = assign_breaks(driver, start, end, "15-min")

            # Assign schedule and breaks for this day
            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))




# Allocate 12-hour drivers with a fixed set per bus
def allocate_12hr_drivers():
    global driver_count
    rest_day_counter = {}

    # Step 1: Assign a fixed set of 12-hour drivers for each bus
    for bus_index, bus in enumerate(buses):
        # First four buses get one pool of 12-hour drivers
        if bus_index < 4:  
            for i in range(3):  # Assign 3 drivers per bus in the first group
                driver_count += 1
                driver = Driver(f"Driver_{driver_count}_12hr", "12-hour")
                bus.drivers.append(driver)  # Attach driver to this bus
                drivers.append(driver)  # Add to global driver list
                rest_day_counter[driver.driver_id] = 0  # Initialize rest days
        else:  # Remaining four buses get another pool of 12-hour drivers
            for i in range(3):  # Assign 3 drivers per bus in the second group
                driver_count += 1
                driver = Driver(f"Driver_{driver_count}_12hr", "12-hour")
                bus.drivers.append(driver)  # Attach driver to this bus
                drivers.append(driver)  # Add to global driver list
                rest_day_counter[driver.driver_id] = 0  # Initialize rest days

    # Step 2: Assign shifts per bus, ensuring one driver works per day with staggered times
    for bus in buses:
        # Rotate through drivers for each bus based on rest-day counter
        driver_index = 0  # To rotate among the drivers assigned to the bus

        for day in WEEKDAYS + WEEKENDS:
            # Find the next available driver (with rest_day_counter == 0)
            while rest_day_counter[bus.drivers[driver_index].driver_id] > 0:
                rest_day_counter[bus.drivers[driver_index].driver_id] -= 1
                driver_index = (driver_index + 1) % len(bus.drivers)  # Move to the next driver

            # Get the selected driver
            driver = bus.drivers[driver_index]

            # Determine start time for weekday shifts
            if day in WEEKDAYS:
                if bus.bus_index < 4:  # First group of buses
                    staggered_start_time = (datetime.strptime("14:00", "%H:%M") + timedelta(minutes=bus.bus_index * 15)).strftime("%H:%M")
                else:  # Second group of buses
                    staggered_start_time = (datetime.strptime("17:00", "%H:%M") + timedelta(minutes=(bus.bus_index - 4) * 15)).strftime("%H:%M")
            else:  # Weekend shifts start at 06:00
                staggered_start_time = (datetime.strptime("06:00", "%H:%M") + timedelta(minutes=bus.bus_index * 18)).strftime("%H:%M")


            # Create time blocks for the shift
            start, end = create_time_blocks(staggered_start_time, SHIFT_12_HOURS)

            # Assign breaks based on shift type
            if day in WEEKDAYS + ["Saturday", "Sunday"]:  # Day shifts (including weekends) get lunch breaks
                if start.hour == 6 and start.minute == 0:  # Only the driver starting at 06:00 gets a 1-hour lunch break
                    breaks = assign_breaks(driver, start, end, "1-hour")
                else:  # For other cases
                    breaks = assign_breaks(driver, start, end, "15-min")
            else:  # Night shifts (including weekends) get 15-minute breaks
                breaks = assign_breaks(driver, start, end, "15-min")

            # Add breaks and schedule to the driver and bus
            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

            # Update rest_day_counter for the selected driver
            rest_day_counter[driver.driver_id] = 2  # Set to rest for 2 days

            # Move to the next driver for the following day
            driver_index = (driver_index + 1) % len(bus.drivers)



# Extend the driver pool to include weekend-only 12-hour drivers
def allocate_weekend_12hr_drivers():
    global driver_count

    # Weekend-only drivers for the night shift
    for bus_index, bus in enumerate(buses):
        # Add two drivers per bus: one for Saturday and one for Sunday
        for day in WEEKENDS:
            driver_count += 1
            driver = Driver(f"Driver_{driver_count}_12hr", f"12-hour (Weekend {day})")
            bus.drivers.append(driver)  # Add to the bus's driver list
            drivers.append(driver)  # Add to the global driver list

            # Assign shifts for the weekend driver
            start_time = (datetime.strptime("18:03", "%H:%M") + timedelta(minutes=bus.bus_index * 18)).strftime("%H:%M")
            start, end = create_time_blocks(start_time, SHIFT_12_HOURS)

            # Assign 15-minute breaks
            breaks = assign_breaks(driver, start, end, "15-min")
            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))


# Assign drivers
allocate_8hr_drivers()
allocate_12hr_drivers()
allocate_weekend_12hr_drivers()

# Output the schedule in the requested format
print("Final Bus Schedule:")
for bus in buses:
    print(f"\n{bus.bus_id} Weekly Schedule:")
    for day in WEEKDAYS + WEEKENDS:
        if bus.schedule[day]:
            print(f"  {day}:")
            for shift in bus.schedule[day]:
                start_time = shift[0].strftime("%H:%M")
                end_time = shift[1].strftime("%H:%M")
                driver_id = shift[2]
                print(f"    Time: {start_time} - {end_time}, Driver: {driver_id}")

print("\nFinal Driver Schedule:")
for driver in drivers:
    print(f"\n{driver.driver_id} ({driver.shift_type}) Weekly Schedule:")
    for day in WEEKDAYS + WEEKENDS:
        if driver.schedule[day]:
            print(f"  {day}:")
            for shift in driver.schedule[day]:
                start_time = shift[0].strftime("%H:%M")
                end_time = shift[1].strftime("%H:%M")
                bus_id = shift[2]
                print(f"    Time: {start_time} - {end_time}, Bus: {bus_id}")
            if driver.breaks[day]:
                print("    Breaks:")
                for b in driver.breaks[day]:
                    print(f"      {b[0]} - {b[1]}")

print(f"\nTotal Drivers Used: {driver_count}")


Final Bus Schedule:

Bus_1 Weekly Schedule:
  Monday:
    Time: 06:00 - 14:00, Driver: Driver_1_8hr
    Time: 14:00 - 02:00, Driver: Driver_9_12hr
  Tuesday:
    Time: 06:00 - 14:00, Driver: Driver_1_8hr
    Time: 14:00 - 02:00, Driver: Driver_10_12hr
  Wednesday:
    Time: 06:00 - 14:00, Driver: Driver_1_8hr
    Time: 14:00 - 02:00, Driver: Driver_11_12hr
  Thursday:
    Time: 06:00 - 14:00, Driver: Driver_1_8hr
    Time: 14:00 - 02:00, Driver: Driver_9_12hr
  Friday:
    Time: 06:00 - 14:00, Driver: Driver_1_8hr
    Time: 14:00 - 02:00, Driver: Driver_10_12hr
  Saturday:
    Time: 06:00 - 18:00, Driver: Driver_11_12hr
    Time: 18:03 - 06:03, Driver: Driver_33_12hr
  Sunday:
    Time: 06:00 - 18:00, Driver: Driver_9_12hr
    Time: 18:03 - 06:03, Driver: Driver_34_12hr

Bus_2 Weekly Schedule:
  Monday:
    Time: 06:15 - 14:15, Driver: Driver_2_8hr
    Time: 14:15 - 02:15, Driver: Driver_12_12hr
  Tuesday:
    Time: 06:15 - 14:15, Driver: Driver_2_8hr
    Time: 14:15 - 02:15, Driver: D

#### 2. Displaying output in excel file (for VLOB)

In [41]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from openpyxl.styles import Font

def extract_break_data(drivers):
    """
    Extract lunch and short break data from driver.breaks dynamically.
    Only include drivers with relevant breaks in the output.
    """
    lunch_breaks_data = {"Driver": []}
    short_breaks_data = {"Driver": []}

    # Initialize the days of the week
    for day in WEEKDAYS + WEEKENDS:
        lunch_breaks_data[day] = []
        short_breaks_data[day] = []

    # Loop through drivers
    for driver in drivers:
        # Collect lunch and short breaks for this driver
        lunch_breaks_per_day = []
        short_breaks_per_day = []

        for day in WEEKDAYS + WEEKENDS:
            # Extract breaks for lunch vs short based on break type
            lunch_breaks = [
                f"{b[0]}-{b[1]}" for b in driver.breaks[day] if b[2] == "1-hour"
            ]
            short_breaks = [
                f"{b[0]}-{b[1]}" for b in driver.breaks[day] if b[2] == "15-min"
            ]

            # Append breaks or empty strings for this day
            lunch_breaks_per_day.append("; ".join(lunch_breaks) if lunch_breaks else "")
            short_breaks_per_day.append("; ".join(short_breaks) if short_breaks else "")

        # Include the driver in lunch data only if they have any lunch breaks
        if any(lunch_breaks_per_day):
            lunch_breaks_data["Driver"].append(driver.driver_id)
            for day, breaks in zip(WEEKDAYS + WEEKENDS, lunch_breaks_per_day):
                lunch_breaks_data[day].append(breaks)

        # Include the driver in short break data only if they have any short breaks
        if any(short_breaks_per_day):
            short_breaks_data["Driver"].append(driver.driver_id)
            for day, breaks in zip(WEEKDAYS + WEEKENDS, short_breaks_per_day):
                short_breaks_data[day].append(breaks)

    # Convert extracted data to DataFrames
    lunch_breaks_df = pd.DataFrame(lunch_breaks_data)
    short_breaks_df = pd.DataFrame(short_breaks_data)

    return lunch_breaks_df, short_breaks_df



# Extract breaks dynamically
lunch_breaks_df, short_breaks_df = extract_break_data(drivers)

# Prepare the bus-to-driver grouped schedule
grouped_schedule = []  # To store grouped schedules by bus

for bus in buses:
    # Add the bus row as a header
    bus_row = [str(bus.bus_id)] + [""] * (len(WEEKDAYS + WEEKENDS))
    grouped_schedule.append(bus_row)

    # Add rows for each driver assigned to the bus
    unique_drivers = set()
    for day in WEEKDAYS + WEEKENDS:
        for _, _, driver_id in bus.schedule.get(day, []):
            unique_drivers.add(driver_id)

    for driver_id in sorted(unique_drivers):  # Ensure consistent ordering
        driver_row = [str(driver_id)]
        for day in WEEKDAYS + WEEKENDS:
            # Aggregate all shifts for this driver on the current day
            shifts = [
                f"{start.strftime('%H:%M')} - {end.strftime('%H:%M')}"
                for start, end, driver in bus.schedule.get(day, [])
                if driver == driver_id
            ]
            driver_row.append(", ".join(shifts) if shifts else "")
        grouped_schedule.append(driver_row)

# Convert grouped schedule to DataFrame
columns = ["Driver"] + WEEKDAYS + WEEKENDS
bus_schedule_df = pd.DataFrame(grouped_schedule, columns=columns)


# Restructure breaks for Lunch and Short Breaks
lunch_breaks_filtered = lunch_breaks_df[lunch_breaks_df.drop(columns=["Driver"]).notna().any(axis=1)]
short_breaks_filtered = short_breaks_df[short_breaks_df.drop(columns=["Driver"]).notna().any(axis=1)]

lunch_rows = [["Lunch Break"] + [""] * (len(columns) - 1)] + lunch_breaks_filtered.values.tolist()
short_break_rows = [["Short Breaks"] + [""] * (len(columns) - 1)] + short_breaks_filtered.values.tolist()
final_breaks_rows = lunch_rows + short_break_rows
breaks_final_df = pd.DataFrame(final_breaks_rows, columns=columns)

# Save DataFrame to an Excel file
file_path = r"Driver_Schedule.xlsx"
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    bus_schedule_df.to_excel(writer, sheet_name="Driver Timetable", index=False)
    breaks_final_df.to_excel(writer, sheet_name="Breaks Schedule", index=False)

# Open the Excel file using openpyxl
wb = load_workbook(file_path)



# Define a function to make row headers bold
def make_row_headers_bold(sheet, header_rows):
    for row_idx in header_rows:
        for cell in sheet[row_idx]:
            cell.font = Font(bold=True)


# Apply bold formatting to row headers
for sheet_name in wb.sheetnames:
    sheet = wb[sheet_name]
    
    # Determine header rows based on the table
    header_rows = []
    for row_idx, row in enumerate(sheet.iter_rows(min_row=1, max_row=sheet.max_row, values_only=True), start=1):
        if row[0] and (row[0].startswith("Bus_") or row[0] in ["Lunch Break", "Short Breaks"]):  # Identify row headers
            header_rows.append(row_idx)

    make_row_headers_bold(sheet, header_rows)
   
# Define a function to adjust column width and enable text wrapping
def auto_adjust_columns(sheet):
    for col in sheet.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            try:
                if cell.value and len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = max_length + 2  # Adding some extra padding
        sheet.column_dimensions[column].width = adjusted_width
        
        # Apply wrap text formatting to all cells
        for cell in col:
            cell.alignment = Alignment(wrap_text=True)


# Apply formatting to each sheet
for sheet_name in wb.sheetnames:
    sheet = wb[sheet_name]
    auto_adjust_columns(sheet)

# Save the modified workbook
wb.save(file_path)

print(f"Excel file saved with bold row headers: {file_path}")

print("Driver Timetable:\n", bus_schedule_df)
print("\nBreaks Schedule:\n", breaks_final_df)


Excel file saved with bold row headers: Driver_Schedule.xlsx
Driver Timetable:
             Driver         Monday        Tuesday      Wednesday  \
0            Bus_1                                                
1   Driver_10_12hr                 14:00 - 02:00                  
2   Driver_11_12hr                                14:00 - 02:00   
3     Driver_1_8hr  06:00 - 14:00  06:00 - 14:00  06:00 - 14:00   
4   Driver_33_12hr                                                
5   Driver_34_12hr                                                
6    Driver_9_12hr  14:00 - 02:00                                 
7            Bus_2                                                
8   Driver_12_12hr  14:15 - 02:15                                 
9   Driver_13_12hr                 14:15 - 02:15                  
10  Driver_14_12hr                                14:15 - 02:15   
11    Driver_2_8hr  06:15 - 14:15  06:15 - 14:15  06:15 - 14:15   
12  Driver_35_12hr                               